# Packing circuits

This notebook looks at some examples of packing circuits to reduce their ancilla costs. In particular in these examples this problem is solved via the calculation of minimum vertex covers, which is done by not allowing 'evicted gates'.

First, import the modules we need.

In [1]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket_dqc.packing import BipartiteCircuit
from pytket_dqc.networks import NISQNetwork
from pytket_dqc.distributors import GraphPartitioning, Annealing
from pytket_dqc.circuits import DistributedCircuit
from pytket.transform import Transform

## Trivial example

The first circuit we'll look at is extremely trivial, we have two global gates which we can pack together with one ebit.

In [2]:
circuit = Circuit()

server_1 = circuit.add_q_register('Server 1', 1)
server_2 = circuit.add_q_register('Server 2', 1)

circuit.CZ(server_1[0],server_2[0])
circuit.CZ(server_1[0],server_2[0])

render_circuit_jupyter(circuit)

In [3]:
bpcircuit = BipartiteCircuit(circuit)
print(f'We can pack this with {bpcircuit.get_ebit_cost()} ebits.')
render_circuit_jupyter(bpcircuit.packed_circuit)

We can pack this with 1 ebits.


## (Anti) Diagonal local gates

Even if we have local gates inserted between our controls, if the local gate is antidiagonal/diagonal, we can still pack these together.

In [4]:
circuit = Circuit()

server_1 = circuit.add_q_register('Server 1', 1)
server_2 = circuit.add_q_register('Server 2', 1)

circuit.CZ(server_1[0],server_2[0]).X(server_1[0]).X(server_2[0]).CZ(server_1[0],server_2[0])

render_circuit_jupyter(circuit)

X is antidiagonal, so does not affect our bipartite graph.

In [5]:
bpcircuit = BipartiteCircuit(circuit)
print(f'We can pack this with {bpcircuit.get_ebit_cost()} ebits.')
render_circuit_jupyter(bpcircuit.packed_circuit)

We can pack this with 1 ebits.


## More complicated circuits

Let's up the ante a bit now, and look at a circuit across two servers, each with 3 qubits.

In [6]:
circuit = Circuit()

s1 = circuit.add_q_register('Server 1', 3)
s2 = circuit.add_q_register('Server 2', 3)

circuit.CX(s1[0], s2[0]).CX(s1[1], s2[0]).CX(s1[1], s2[1]).H(s1[1]).CX(s1[1], s2[2]).CX(s1[2], s2[1]).CX(s1[0], s2[0]).CX(s1[0], s2[1]).H(s2[0]).CX(s1[0], s2[2]).X(s1[0]).CX(s1[0], s2[0])

render_circuit_jupyter(circuit)

Just by looking at this circuit, it's not immediately obvious what the minimum number of ebits required is.

Luckily `BipartiteCircuit.get_ebit_cost()` can tell us!

In [7]:
bpcircuit = BipartiteCircuit(circuit)
print(f'We can pack this with {bpcircuit.get_ebit_cost()} ebits.')
render_circuit_jupyter(bpcircuit.packed_circuit)

We can pack this with 4 ebits.


In [11]:
with open('circuits/uccsd/raw/20_02_01.json') as f:
    circuit_s = json.load(f)
circuit = Circuit.from_dict(circuit_s)
Transform.RebaseToQuil().apply(circuit)

network = NISQNetwork([[0,1], [1,2]], {0:[0, 1, 2, 3], 1:[4, 5, 6], 2:[7, 8, 9]})
distributor = GraphPartitioning()

def distribute_circuit_with_bipartite_packing(circuit, network, distributor):
    dist_circ = DistributedCircuit(circuit)
    placement = distributor.distribute(dist_circ, network)
    circ_with_dist = dist_circ.to_relabeled_registers(placement)
    bpcircuit = BipartiteCircuit(circ_with_dist)
    print(f'Ebit cost is {bpcircuit.get_ebit_cost()}')
    return bpcircuit

bpcircuit = distribute_circuit_with_bipartite_packing(circuit, network, distributor)

Ebit cost is 1008
